In [14]:
import math
import sys,os
import numpy as np
import pandas as pd

from mgwr.gwr import GWR,MGWR
from mgwr.sel_bw import Sel_BW

In [15]:
zillow = pd.read_csv("Zillow-test-dataset/zillow_1k.csv",sep=',')
zillow.head()

,utmX,utmY,value,nbaths,nbeds,area,age
0,374161.333830,3.757568e+06,56.976,1.0,2.0,954.0,71.0
1,412657.822910,3.761734e+06,157.982,2.0,4.0,1614.0,45.0
2,391600.475640,3.782212e+06,512.612,4.0,4.0,2962.0,54.0
3,375056.064681,3.748579e+06,111.437,2.0,3.0,1292.0,65.0
4,388693.693038,3.779865e+06,128.327,1.0,3.0,1169.0,78.0


In [16]:
#Converting things into matrices
y = zillow.value.values.reshape(-1,1)
X = zillow.iloc[:,3:].values
k = zillow.shape[1]
u = zillow.utmX
v = zillow.utmY
n = zillow.shape[0]
coords = np.array(list(zip(u,v)))

In [17]:
X = (X - np.mean(X,axis=0))/np.std(X, axis=0)
y = (y - np.mean(y,axis=0))/np.std(y, axis=0)

### `mgwr`

In [18]:
%%time
#Bandwidth searching
selector = Sel_BW(coords,y,X,multi=True)
bws = selector.search(verbose=True)
print("Optimal Bandwidths:",bws)

#Fitting the model with optimal bandwidth
pysal_result_mgwr=MGWR(coords,y,X,selector=selector).fit()

Current iteration: 1 ,SOC: 0.0054557
Bandwidths: 133.0, 52.0, 116.0, 82.0, 50.0
Current iteration: 2 ,SOC: 0.0044876
Bandwidths: 158.0, 105.0, 120.0, 55.0, 50.0
Current iteration: 3 ,SOC: 0.0026065
Bandwidths: 182.0, 127.0, 122.0, 55.0, 43.0
Current iteration: 4 ,SOC: 0.0022962
Bandwidths: 202.0, 195.0, 122.0, 55.0, 43.0
Current iteration: 5 ,SOC: 0.0016683
Bandwidths: 202.0, 285.0, 125.0, 55.0, 43.0
Current iteration: 6 ,SOC: 0.0010319
Bandwidths: 202.0, 365.0, 125.0, 55.0, 43.0
Current iteration: 7 ,SOC: 0.0013286
Bandwidths: 202.0, 999.0, 125.0, 55.0, 43.0
Current iteration: 8 ,SOC: 0.0004091
Bandwidths: 202.0, 999.0, 125.0, 55.0, 43.0
Current iteration: 9 ,SOC: 0.0001911
Bandwidths: 202.0, 999.0, 125.0, 55.0, 43.0
Current iteration: 10 ,SOC: 0.0001056
Bandwidths: 202.0, 999.0, 125.0, 55.0, 43.0
Current iteration: 11 ,SOC: 6.57e-05
Bandwidths: 202.0, 999.0, 125.0, 55.0, 43.0
Current iteration: 12 ,SOC: 4.4e-05
Bandwidths: 202.0, 999.0, 125.0, 55.0, 43.0
Current iteration: 13 ,SOC: 3


CPU times: user 6min 35s, sys: 4.31 s, total: 6min 39s
Wall time: 1min 40s


### FastGWR

In [63]:
#Running FastGWR 4 Processor
!fastgwr run -np 4 -data Zillow-test-dataset/zillow_1k.csv -out results_mgwr.csv -adaptive -constant -mgwr



------------------------------------------------------------
Starting FastGWR with 4 Processors
Spatial Kernel: Adaptive Bisquare
Data Input Path: Zillow-test-dataset/zillow_1k.csv
Output Result Path: results_mgwr.csv
Constant: True
MGWR Backfitting...
Data are standardized
Initialization Done...
Iter: 1 SOC: 5.46e-03
bws: [133.0, 52.0, 116.0, 82.0, 50.0]
Iter: 2 SOC: 4.49e-03
bws: [158.0, 105.0, 120.0, 55.0, 50.0]
Iter: 3 SOC: 2.61e-03
bws: [182.0, 127.0, 122.0, 55.0, 43.0]
Iter: 4 SOC: 2.30e-03
bws: [202.0, 195.0, 122.0, 55.0, 43.0]
Iter: 5 SOC: 1.67e-03
bws: [202.0, 285.0, 125.0, 55.0, 43.0]
Iter: 6 SOC: 1.03e-03
bws: [202.0, 365.0, 125.0, 55.0, 43.0]
Iter: 7 SOC: 1.33e-03
bws: [202.0, 999.0, 125.0, 55.0, 43.0]
Iter: 8 SOC: 4.09e-04
bws: [202.0, 999.0, 125.0, 55.0, 43.0]
Iter: 9 SOC: 1.91e-04
bws: [202.0, 999.0, 125.0, 55.0, 43.0]
Iter: 10 SOC: 1.06e-04
bws: [202.0, 999.0, 125.0, 55.0, 43.0]
Iter: 11 SOC: 6.57e-05
bws: [202.0, 999.0, 125.0, 55.0, 43.0]
Iter: 12 SOC: 4.40e-05
bws: [2

### Results validation

In [77]:
print("MGWR pysal AICc - ", pysal_result_mgwr.aicc)
print("MGWR pysal R2 - ",pysal_result_mgwr.R2)
print("MGWR pysal ENP_j - ",pysal_result_mgwr.ENP_j)

MGWR pysal AICc -  1679.0150747382786
MGWR pysal R2 -  0.7660681409024748
MGWR pysal ENP_j -  [10.13917587  1.15749374 16.97300011 42.38091109 56.32981658]


In [71]:
fastGWR_result_mgwr = pd.read_csv("results_mgwr.csv")
fastGWR_result_mgwr.head()

,index,residual,b_intercept,b_nbaths,b_nbeds,b_area,b_age,se_intercept,se_nbaths,se_nbeds,se_area,se_age
0,0.0,-0.011031,-0.075463,0.109868,-0.101711,0.496562,-0.152488,0.052884,0.038352,0.067540,0.153287,0.140887
1,1.0,0.240776,-0.149862,0.102825,-0.061923,0.483481,0.015166,0.055685,0.037990,0.072307,0.123264,0.121128
2,2.0,0.879991,-0.022973,0.107025,0.003100,0.368938,-0.133318,0.050438,0.038407,0.067751,0.145816,0.078188
3,3.0,0.096153,-0.132249,0.109213,-0.179828,0.547619,0.045308,0.053338,0.038351,0.075962,0.151626,0.185558
4,4.0,0.188585,-0.028293,0.107749,-0.014474,0.279898,-0.134211,0.052219,0.038482,0.069745,0.142513,0.077780


In [66]:
#Validate model residuals
print(np.allclose(fastGWR_result_mgwr.residual,
                  pysal_result_mgwr.resid_response.reshape(-1)))


True


In [67]:
#Validate parameter estimates
print(np.allclose(np.array(fastGWR_result_mgwr.iloc[:,2:7]),
                  pysal_result_mgwr.params))



True


In [76]:
#Validate parameter estimates standard errors
print(np.allclose(np.array(fastGWR_result_mgwr.iloc[:,7:12]),
                  pysal_result_mgwr.bse,atol=1e-3))


True
